In [57]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, normalize_run, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
import pandas as pd
from tqdm import tqdm
from statistics import mean
from ranx import fuse, Run

In [58]:
ensure_pyterrier_is_loaded()
tira = Client()


In [59]:
LONGEVAL_LAGS = {
    'lag1': {
        'tira_id': 'longeval-2023-01-20240423-training',
        'pt_dataset': pt.get_dataset('irds:ir-lab-padua-2024/longeval-2023-01-20240426-training'),
    },
    'lag6': {
        'tira_id': 'longeval-2023-06-20240418-training',
        'pt_dataset': pt.get_dataset('irds:ir-lab-padua-2024/longeval-2023-06-20240422-training'),
    },
    'lag8': {
        'tira_id': 'longeval-2023-08-20240418-training',
        'pt_dataset': pt.get_dataset('irds:ir-lab-padua-2024/longeval-2023-08-20240422-training'),
    }
}

In [55]:
def to_ranx_dict(run):
    ret = {}
    for _, row in run.iterrows():
        if row['qid'] not in ret:
            ret[row['qid']] = {}

        ret[row['qid']][row['docno']] = row['score']

    return Run(ret)

def from_ranx(run):
    ret = []

    for qid in run.keys():
        for docno in run[qid].keys():
            ret += [{
                'qid': qid,
                'docno': docno,
                'score': run[qid][docno]
            }]
    return pd.DataFrame(ret)

In [77]:
def get_df_with_sparse_cross_encoder_passage_scores(dataset_id, aggregation, system_name):
    predictions = tira.pd.from_retriever_submission('ir-benchmarks/fschlatt/abstract-metaball', dataset_id)
    ret = {}

    for _, row in predictions.iterrows():
        qid = row['qid'].split('___split')[0]
        docno = row['docno'].split('___p')[0]
        score = row['score']
        if qid not in ret:
            ret[qid] = {}
        if docno not in ret[qid]:
            ret[qid][docno] = []
        
        ret[qid][docno] += [score]
    
    ret_df = []
    for qid in ret.keys():
        for docno in ret[qid].keys():
            score = aggregation(ret[qid][docno])
            ret_df += [{
                'qid': qid,
                'docno': docno,
                'score': score,
                'system': system_name
            }]
    return normalize_run(pd.DataFrame(ret_df), system_name)

for lag in tqdm(LONGEVAL_LAGS.keys()):
    system = 'max-cross-encoder-passage-score'
    df_1 = get_df_with_sparse_cross_encoder_passage_scores(LONGEVAL_LAGS[lag]['tira_id'], max, system)
    #df_1 = tira.pd.from_retriever_submission('ir-benchmarks/tira-ir-starter/ColBERT Re-Rank (tira-ir-starter-pyterrier)', LONGEVAL_LAGS[lag]['tira_id'])
    df_2 = tira.pd.from_retriever_submission('workshop-on-open-web-search/fschlatt/rank-zephyr', LONGEVAL_LAGS[lag]['tira_id'])
    df_2 = normalize_run(df_2, system)

    #df_3 = tira.pd.from_retriever_submission('ir-benchmarks/tira-ir-starter/MonoT5 Base (tira-ir-starter-gygaggle)', LONGEVAL_LAGS[lag]['tira_id'])
    

    df = fuse(runs=[to_ranx_dict(df_1), to_ranx_dict(df_2)], method="rrf")
    df = from_ranx(df)
    normalize_run(df, system).to_csv(f'{system}.{lag}', sep=" ", header=False, index=False)

100%|██████████| 3/3 [01:52<00:00, 37.61s/it]


In [69]:
def sec(a):
    return sorted(a, reverse=True)[:2][-1]